# CluedIn

[cluedin](https://pypi.org/project/cluedin/) is a Python client for [CluedIn](https://www.cluedin.com/) API. It can help you with the following:

* getting JWT access tokens to CluedIn
* calling CluedIn REST API
* calling CluedIn GraphQL API

The use is pretty simple. First, install it from [PyPi](https://pypi.org/project/cluedin/):

In [14]:
%pip install cluedin
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


Next, you will need to obtain an access token:

In [15]:
import cluedin
context = {
    "protocol": "http", # default - `https`
    "domain": "cluedin.local:8888",
    "organization": "foobar",
    "user": "admin@foobar.com",
    "password": "Foobar23!"
}

cluedin.load_token_into_context(context)

context

{'protocol': 'http',
 'domain': 'cluedin.local:8888',
 'organization': 'foobar',
 'user': 'admin@foobar.com',
 'password': 'Foobar23!',
 'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IkVDMDMxODBDMDgyMTZEOTIyMjlEMUNDMDM3QUEzODE3IiwidHlwIjoiYXQrand0In0.eyJuYmYiOjE2ODM1NDMwNDEsImV4cCI6MTY4MzU0NjY0MSwiaXNzIjoiaHR0cDovL2NsdWVkaW4ubG9jYWw6OTAwMS8iLCJhdWQiOiJTZXJ2ZXJBcGlGb3JVSSIsImNsaWVudF9pZCI6ImZvb2JhciIsInN1YiI6ImE4MTU2NjhiLTdkODktNGVkZC04ZTBjLThiNzJjMDFkYWEyOCIsImF1dGhfdGltZSI6MTY4MzU0MzA0MSwiaWRwIjoibG9jYWwiLCJyb2xlIjoiT3JnYW5pemF0aW9uQWRtaW4iLCJuYW1lIjoiYWRtaW5AZm9vYmFyLmNvbSIsIk9yZ2FuaXphdGlvbklkIjoiOGUzMjYzOGYtYzdhZC00ZGY2LThhMjEtNzkxMDUxMzkzNzBmIiwiQ2xpZW50SWQiOiJmb29iYXIiLCJPcmdhbml6YXRpb25OYW1lIjoiZm9vYmFyIiwiT3JnYW5pemF0aW9uQ2xpZW50UmVmZXJlbmNlIjoiZm9vYmFyIiwiSWQiOiJhODE1NjY4Yi03ZDg5LTRlZGQtOGUwYy04YjcyYzAxZGFhMjgiLCJSb2xlQ2xhaW1zIjoie1wiZ292ZXJuYW5jZS5zdWJqZWN0YWNjZXNzcmVxdWVzdFwiOlwiQ29uc3VsdGFudFwiLFwiZ292ZXJuYW5jZS5wZXJzb25hbGlkZW50aWZpZXJzXCI6XCJDb25zdWx0YW50XCIsXCJlbmdpbmUuY2

Run a GraphQL query:

In [16]:
query = """
    query searchEntities($cursor: PagingCursor, $query: String, $pageSize: Int) {
      search(query: $query, sort: DATE, cursor: $cursor, pageSize: $pageSize) {
        totalResults
        cursor
        entries {
          id
          name
          entityType
        }
      }
    }
"""

variables = {
    "query": "entityType:/Infrastructure/User",
    "pageSize": 1
}

response = cluedin.gql.gql(context, query, variables)

response

{'data': {'search': {'totalResults': 1,
   'cursor': 'ewAiAFAAYQBnAGUAIgA6ADEALAAiAFAAYQBnAGUAUwBpAHoAZQAiADoAMQAsACIAQwBvAG0AcABvAHMAaQB0AGUAQQBmAHQAZQByACIAOgB7AH0ALAAiAFMAZQBhAHIAYwBoAEEAZgB0AGUAcgAiADoAWwAxADYAOAAzADUANAAyADAAOQAyADIAOAAzACwAMQA2ADgAMwA1ADQAMgAwADkAMgAyADgAMwAsADEANgA4ADMANQA0ADIAMAA5ADIAMgA4ADMAXQB9AA==',
   'entries': [{'id': '24ee44b4-3d42-50ea-8911-5cda4fe1e64f',
     'name': 'admin@foobar.com',
     'entityType': '/Infrastructure/User'}]}}}

Get paged results:

In [17]:
import numpy as np
import pandas as pd

query = """
    query searchEntities($cursor: PagingCursor, $query: String, $pageSize: Int) {
      search(query: $query, sort: DATE, cursor: $cursor, pageSize: $pageSize) {
        totalResults
        cursor
        entries {
          id
          name
          entityType
        }
      }
    }
"""

variables = {
    "query": "*",
    "pageSize": 10000
}

entries = np.array([x for x in cluedin.gql.entries(context, query, variables)])

df = pd.DataFrame(entries.tolist(), columns=list(entries[0].keys()))
df

,id,name,entityType
0,dc60b862-9f3d-5ad6-91e8-4dc1c2c8b41f,Clarance Laxson,/Person
1,7b24617a-eb1a-51a7-af04-66bce2307c82,Phillipe Scola,/Person
2,b64b7e9b-1d21-522f-8a92-7f0929fc7e00,Raul Oloshkin,/Person
3,d79bebdf-84bc-5f40-9d64-4a546aeee41c,Celle Haycroft,/Person
4,a6a6847e-a85c-53c7-a451-65c155dbe830,Emelyne Margrem,/Person
...,...,...,...
929,ca2fbf84-78c1-537a-a3c1-1db5d7e6ff04,Duplicate Email,/Person
930,e83546c1-73bb-58fe-82cc-4e8e1662ed5b,AcceptanceCrawler,/Provider/Root
931,e10887dc-0e81-5979-a0c1-1146d5c34137,Child Person,/Person
932,24ee44b4-3d42-50ea-8911-5cda4fe1e64f,admin@foobar.com,/Infrastructure/User
